In [1]:
## prepare InSAR stacks for the area of interest (AOI) for PySAR from the full track dataset from ISCE
## 1. Load the whole dataset into HDF5
## 2. geocode the stack with desired spatial resolution and coverage

%matplotlib inline
import os
from pysar.utils import readfile, utils as ut

proj_basename = 'Kirishima'
proj_dir = os.path.join(os.path.expanduser('~/insarlab'), proj_basename)
dem_file = os.path.join(proj_dir, 'DEM/gsi10m.dem.wgs84')

## AOI setup
AOIs = []
AOI = {}
AOI['name'] = 'Shinmoedake'
AOI['SNWE'] = '31.88 31.94 130.85 130.91'
AOI['step'] = '0.0002777777777777778'
AOIs.append(AOI)
AOI = {}
AOI['name'] = 'Kirishima'
AOI['SNWE'] = '31.7 32.1 130.5 131.1'
AOI['step'] = '0.000925925925925926'
AOIs.append(AOI)

In [2]:
def extract_dataset_lalo(AOI, src_dir, dst_dir, dem_file=None, update_mode=True):
    #location and SNWE of AOI
    if not os.path.isdir(dst_dir):
        os.makedirs(dst_dir)
        print('create directory', dst_dir)

    lut_file = os.path.join(src_dir, 'INPUTS/geometryRadar.h5')
    opt = ' -l {l} -x {s} -y -{s} --bbox {b} '.format(l=lut_file, s=AOI['step'], b=AOI['SNWE'])

    # get geometry
    src_file = os.path.join(src_dir, 'INPUTS/geometryRadar.h5')
    dst_file = os.path.join(dst_dir, 'geometryGeo.h5')
    if not update_mode or ut.run_or_skip(in_file=src_file, out_file=dst_file) == 'run':
        !geocode.py $src_file -o $dst_file $opt

    # get ifgramStack
    src_file = os.path.join(src_dir, 'INPUTS/ifgramStack.h5')
    dst_file = os.path.join(dst_dir, 'ifgramStack.h5')
    if not update_mode or ut.run_or_skip(in_file=src_file, out_file=dst_file) == 'run':
        !geocode.py $src_file -o $dst_file $opt

    # prep original resolution DEM
    if dem_file:
        dem_dir = os.path.dirname(dem_file)
        dst_file = os.path.join(dem_dir, '{}.dem.wgs84'.format(AOI['name']))
        if not update_mode and ut.run_or_skip(in_file=dem_file, out_file=dst_file) == 'run':
            S, N, W, E = AOI['SNWE'].split()
            opt = '-l {} {} -L {} {}'.format(S, N, W, E)
            !subset.py $dem_file -o $dst_file $opt

In [4]:
## 2.0 Source files - InSAR stacks in radar coordinates
proj_names = ['KirishimaAlosAT424F620_630', 'KirishimaAlosDT73F2970_2980']

for proj_name in proj_names:
    dset_name = proj_name.split(proj_basename)[1]
    src_dir = os.path.join(proj_dir, dset_name, 'PYSAR')
    os.chdir(src_dir)
    print('Go to directory', src_dir)

    temp_file = os.path.expandvars('$TE/{}.template'.format(proj_name))
    !pysarApp.py $temp_file --dostep load_data

    # check spatial resolution - for the reference of geocoding post size
    atr = readfile.read_attribute('./INPUTS/ifgramStack.h5')
    print('range ground resolution: {} m'.format(ut.range_ground_resolution(atr)))
    print('azimuth ground resolution: {} m'.format(ut.azimuth_ground_resolution(atr)))

    dset_basename = dset_name.split('F')[0]
    for AOI in AOIs:
        dst_dir = os.path.join(proj_dir, '{}{}'.format(AOI['name'], dset_basename), 'PYSAR/INPUTS')
        extract_dataset_lalo(AOI, src_dir, dst_dir, dem_file)

Go to directory /Users/yunjunz/insarlab/Kirishima/AlosAT424F620_630/PYSAR
PySAR version 1.0.0-dev, release date 2019-03-31
--RUN-at-2019-04-18 13:37:29.595469--
Run routine processing with pysarApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'correct_unwrap_error', 'stack_interferograms', 'invert_network', 'correct_LOD', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--dostep option enabled, disable the plotting at the end of the processing.
--------------------------------------------------
Project name: KirishimaAlosAT424F620_630
Go to work directory: /Users/yunjunz/insarlab/Kirishima/AlosAT424F620_630/PYSAR
['/Users/yunjunz/insarlab/Kirishima/AlosAT424F620_630/PYSAR/INPUTS/KirishimaAlosAT424F620_630.template'] exists and is newer than ['/Users/yunjunz/insarlab/infiles_yzhang1/TEMPLATES/KirishimaAlosAT424F620_630.template'] --> skip.
read custom template f

finished writing to /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/INPUTS/geometryGeo.h5
time used: 00 mins 7.1 secs.

output pixel size in (lat, lon) in degree: (-0.000925925925925926, 0.000925925925925926)
output area extent in (S N W E) in degree: (31.7, 32.1, 130.5, 131.1)
--------------------------------------------------
resampling file: /Users/yunjunz/insarlab/Kirishima/AlosAT424F620_630/PYSAR/INPUTS/ifgramStack.h5
reading coherence        from ifgramStack.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading connectComponent from ifgramStack.h5 ...
restrict fill value to False for bool type source data
nearest resampling with kd_tree (0 segments) ...
reading unwrapPhase      from ifgramStack.h5 ...
nearest resampling with kd_tree (0 segments) ...
create HDF5 file: /Users/yunjunz/insarlab/Kirishima/KirishimaAlosAT424/PYSAR/INPUTS/ifgramStack.h5 with w mode
create dataset /coherence        of float32    in size of (225, 433, 649)      with compression=None
c

reading height             from geometryRadar.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading incidenceAngle     from geometryRadar.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading latitude           from geometryRadar.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading longitude          from geometryRadar.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading shadowMask         from geometryRadar.h5 ...
restrict fill value to False for bool type source data
nearest resampling with kd_tree (0 segments) ...
reading slantRangeDistance from geometryRadar.h5 ...
nearest resampling with kd_tree (0 segments) ...
reading waterMask          from geometryRadar.h5 ...
restrict fill value to False for bool type source data
nearest resampling with kd_tree (0 segments) ...
create HDF5 file: /Users/yunjunz/insarlab/Kirishima/KirishimaAlosDT73/PYSAR/INPUTS/geometryGeo.h5 with w mode
create dataset /azimuthAngle       of float32    in size of (433, 